In [1]:
# Import our packages
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# read from csv file
# read data
df= pd.read_csv('https://raw.githubusercontent.com/Hala-Alsouly/Theatre/main/Theaters.csv')
df.head()

,Branch,Theater,Type,seats,AdultCost,ChildCost,Movie,AgeRate,Shows
0,Riyadh,1,VIP,60,20,15,Limitless,PG-13,3
1,Riyadh,2,Standard,120,15,10,The Old Guard,R,2
2,Riyadh,3,Standard,120,15,10,Raya and the Last Dragon,PG,4
3,Riyadh,4,MAX,200,17,12,Rush Hour 3,PG-13,2
4,Riyadh,5,Standard,120,15,10,Focus,16,1


# 1. Create snacks that the customers can buy and randomize who buys which snack

In [3]:
###################################### Snacks ##########################################

#Create snacks dataframe to store the sales
snacksDF = pd.DataFrame(columns = [])

#Print the data  
print(snacksDF)  

Empty DataFrame
Columns: []
Index: []


In [4]:
################################## Snacks Function #####################################

# the function takes snacks data frame to store the sales on it, Branch name, theater number
# and takes number of visitors.It is return the total revenue of the day for a theater.
def snacksF(snacksDF, BranchName, TheaterN, visitorsN):
    #snacks prices
    prices= [6, 8, 10, 8, 7, 9]
    #create a vector of sold snacks
    snack = random.sample(range(0,visitorsN), 6)
    #calculate the sales of snacks for a screen
    snacksSales= np.array(snack) @ np.array(prices)
    #bind it with the branch and theater
    snack= [BranchName, TheaterN,snack]
    return snacksSales,snack

In [5]:
#Note this will add to the revenue function
sales, snack = snacksF(snacksDF, "BranchName", "TheaterN", 30)

#add it to the dataframe
snacksDF =snacksDF.append({'Branch':snack[0],
                           'Theater':snack[1],
                           'Small Popcorn':snack[2][0],
                           'Medium Popcorn':snack[2][1],
                           'Large Popcorn':snack[2][2],
                           'Icecream':snack[2][3],
                           'Soft Drink':snack[2][4],
                           'Frozen':snack[2][5]}, ignore_index=True)

In [6]:
'''
 This function takes theater dataframe and snack dataframe, then generates random
 number of visitors for each branch, each theatre, each day, each show time.
 It is return a dataframe of the generated info.
'''
def runTheatre(theatredf, snackdf):
    column_names=['Branch', 'Theater', 'Day', 'Show', 'nAdults', 'nChildren', 'nStusents', 'nSenior']
    revenudf= pd.DataFrame(columns = column_names)
    #Iterate through days
    for day in range(1,8):
        #row= np.zeros(6)
        
        #iterate through theatres
        for j in range(len(df)):
            
            #iterae by shows numbers
            for show in range(1,df['Shows'][j]+1):
                row= np.array([df['Branch'][j],df['Theater'][j], day, show ,0,0,0,0], dtype=object)

                # The least visitor accepted is 30
                startSeat=30
                # if it is weekend increase the visiting chance
                if day in range(5,8):
                    startSeat= df['seats'][j] // 2
                adults= np.random.randint(low=startSeat, high=df['seats'][j])
                availableSeats= df['seats'][j] - adults

                #generate students sample from the previous adults
                students= np.random.randint(low=0, high= adults//2)
                row[6]= students

                adults-= students

                #generate senior sample from the previous adult
                senior= np.random.randint(low=0, high= adults//4)
                row[7]= senior

                adults-= senior
                row[4]= adults

                # conditional statements for movies that children are allowed to watch
                if( df['AgeRate'][j] == "PG" or df['AgeRate'][j] == "PG-13"):
                    children= np.random.randint(low=0, high= availableSeats)
                    row[5]= children
                    
                # append the geneated visitors to the dataframe
                revenudf.loc[len(revenudf.index)] = row
                
    return revenudf

# calculate the Discount for senior and students: 10%

In [ ]:
###### return number of students, revenue
def discount (ticketCost,people):
    
    total = ticketCost * people
    # calculate the Discount for senior and students: 10%
    discountedTicket = total * 0.1 
    # calculate the total after apply the Discount 
    total = total - discountedTicket
    
    print("Discount : ",discountedTicket)
    print("Number of people:",people)
    print("Total Revenue  : ",total)
   
    return (discountedTicket,total)
    
    
#a,b = discount(ticketCost,senior)
#a,b = discount(ticketCost,students)
    

# calculate the Discount for sunday: 35%

In [ ]:
def sundayDiscount (ticketCost,people):
    
    total = ticketCost * people
    # calculate the Discount for senior and students: 35%
    discountedTicket = total * 0.35 
    # calculate the total after apply the Discount 
    total = total - discountedTicket
    
    print("Discount : ",discountedTicket)
    print("Number of people:",people)
    print("Total Revenue  : ",total)
   
    return (discountedTicket,total)
    

#a,b = sundayDiscount(ticketCost,senior)
